In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np

# !nbdev_build_lib

take single shot to assess nutrition value of the dish without a need to weight (separately or entirely), search or input


image multiclassificator
- classes
	- limited  number of classes, multiplied by  grid of weights. Something  simple, like "Grilled salmon with  white sauce" * [50 g, 100 g, 150 g, 200 g, 250 g, 300 g]
- calculation
	- KNN search for the image around text vectors. Next, take farthest  neighbours from KNN to pick the different types of food and remove synonyms  
- output
	- something like [Grilled salmon with white sauce 150 g, green salad 100 g]

1 test if weight would work
- to database
- select a few difference types of food, inference weighted weighted
- check accuracy

In [20]:
df = pd.read_csv(data_path/'food.csv')

In [26]:
pdrows(200)

In [39]:
df['Description'] = df['Description'].str.lower()

In [54]:
df[df['Description'].str.contains('cod')&df['Description'].str.contains('')]

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
640,FISH OIL,"fish oil,cod liver",4589,0,0.00,0,0,0.0,570,0,...,0,0,0.00,100000,30000,0.00,0.000,0.0,0.00,0.0
4308,COD,"cod,atlantic,raw",15015,0,1.16,0,0,0.0,43,65,...,413,54,0.45,40,12,0.91,0.245,1.0,0.64,0.1
4309,COD,"cod,atlantic,ckd,dry heat",15016,0,1.49,0,0,0.0,55,84,...,244,78,0.58,47,14,1.05,0.283,1.0,0.81,0.1
4310,COD,"cod,atlantic,cnd,sol&liq",15017,0,1.49,0,0,0.0,55,85,...,528,218,0.58,47,14,1.05,0.282,1.0,0.82,0.1
4311,COD,"cod,atlantic,dried&salted",15018,0,18.67,0,0,0.0,152,291,...,1458,7027,1.59,140,42,10.00,0.864,3.5,2.84,0.4
4312,COD,"cod,pacific,raw",15019,0,1.20,0,0,0.0,37,65,...,403,71,0.40,27,8,0.90,0.400,2.9,0.64,0.1
4338,LINGCOD,"lingcod,raw",15045,0,1.21,0,0,0.0,52,0,...,437,59,0.45,50,15,3.60,0.300,0.0,0.00,0.0
4481,COD,"cod,pacific,ckd,dry heat",15192,0,1.54,0,0,0.0,47,0,...,517,91,0.51,32,10,1.04,0.462,3.0,0.00,0.0
4488,LINGCOD,"lingcod,cooked,dry heat",15199,0,1.55,0,0,0.0,67,0,...,560,76,0.58,58,17,4.15,0.346,0.0,0.00,0.0
7048,FISH,"fish,lingcod,meat,raw (alaska native)",35046,0,1.30,0,0,0.0,0,0,...,433,59,0.00,230,0,0.00,0.000,0.0,0.00,0.0


- test if will better work in less text is used. for banana
1 select all where 'banan' is present
2 inference for only 'product name' short
3 create milvus collection for short and long texts and play with

In [3]:
pdrows(200)

In [7]:
paths = data_path.ls()

In [14]:
n=1

In [21]:
pdrows(10)

In [37]:
p = paths[n]
print(n)
print(p)
n+=1
pd.read_csv(p)

IndexError: list index out of range

In [ ]:
pd.read_sql(data_path)

In [3]:
def get_column_names(Table):
    cols = list(Table.__table__.c)
    return [c.name for c in cols]

create table for babanas

In [4]:
cols = get_column_names(Bananas)
cols = [c for c in cols if not 'clip' in c]

In [5]:
cols

['id',
 'product_name',
 'keywords',
 'ingredients_text',
 'categories',
 'food_groups',
 'energy_kcal_100g',
 'proteins_100g',
 'fat_100g',
 'carbohydrates_100g']

In [6]:
df = pd.read_sql(f'select {",".join(cols)} from foods_big',engine)

In [8]:
df['food_groups'].unique()

array(['ice-cream', 'NaN', 'dressings-and-sauces', 'bread',
       'one-dish-meals', 'meat-other-than-poultry', 'processed-meat',
       'chocolate-products', 'milk-and-yogurt', 'cheese',
       'dairy-desserts', 'unknown', 'fruit-juices', 'eggs', 'legumes',
       'biscuits-and-cakes', 'breakfast-cereals', 'fats', 'sweets',
       'fruits', 'sweetened-beverages', 'vegetables',
       'alcoholic-beverages', 'artificially-sweetened-beverages',
       'lean-fish', 'fruit-nectars', 'salty-and-fatty-products',
       'pizza-pies-and-quiches', 'cereals', 'sandwiches',
       'plant-based-milk-substitutes', 'nuts', 'appetizers',
       'fish-and-seafood', 'dried-fruits', 'soups', 'poultry',
       'teas-and-herbal-teas-and-coffees', 'unsweetened-beverages',
       'offals', 'fruits-and-vegetables', 'pastries', 'potatoes',
       'cereals-and-potatoes', 'waters-and-flavored-waters', 'fatty-fish',
       'fish-meat-eggs'], dtype=object)

In [15]:
df.groupby('food_groups')['product_name'].count().sort_values(ascending=False)

food_groups
NaN                                 683748
biscuits-and-cakes                   44884
sweets                               42842
dressings-and-sauces                 29338
one-dish-meals                       26978
cereals                              26235
cheese                               23317
processed-meat                       19504
milk-and-yogurt                      19103
sweetened-beverages                  16006
fish-and-seafood                     15329
bread                                15170
vegetables                           13849
unsweetened-beverages                13404
chocolate-products                   10489
fats                                  9981
appetizers                            8251
meat-other-than-poultry               7843
fruits                                7753
salty-and-fatty-products              7392
breakfast-cereals                     6909
poultry                               6158
pizza-pies-and-quiches                5993

In [48]:
def bing_search_image(q):
    url = "https://bing-image-search1.p.rapidapi.com/images/search"

    querystring = {"q":q,"count":"1"}

    headers = {
        "X-RapidAPI-Host": "bing-image-search1.p.rapidapi.com",
        "X-RapidAPI-Key": "BI6oP0xGxsmsh6qLhuW9H4zO9gKcp149uMujsnNNKgjMVUdAxg"
    }

    r = requests.request("GET", url, headers=headers, params=querystring)
    return [el['contentUrl'] for el in json.loads(r.text)['value']]

In [50]:
df[df['food_groups']=='vegetables'].sample(10)

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g
454053,672372,Tomate Frito Ecologico,"su, verdura, alimento, tomatoes-and-tomato-pro...",NaN,"Alimentos y bebidas de origen vegetal, Aliment...",vegetables,76.000000,1.500000,3.100000,10.000000
606165,936844,Kelloggs original sausage patties clean eating...,"pattie, and, eating, analogue, sausage, frozen...","advice. Ingredie.nts: Water, wheat gluten, soy...","Plant-based foods and beverages, Plant-based f...",vegetables,210.526316,23.684211,7.894737,13.157895
120358,169267,"Glaser Organic Farms, Cabbage & Jalapeno Sauer...","based, beverage, vegetable, and, plant-based, ...","Organic green cabbage, organic jalapeno, celti...","Plant-based foods and beverages, Plant-based f...",vegetables,42.000000,2.820000,0.000000,8.450000
361623,532840,Macedoine De Legumes Extra 37CL,"base, macedoine, de, canned-drained-diced-mixe...",NaN,"Aliments et boissons à base de végétaux, Alime...",vegetables,159.000000,2.100000,0.300000,5.000000
728464,3546,Chopped Mixed Greens,"beverage, green, based, mixed, kroger, fruit, ...","Turnip greens, mustard greens, water, salt.","Plant-based foods and beverages, Plant-based f...",vegetables,13.000000,0.870000,0.000000,2.610000
151858,214708,Tourte champignons jambon,"verte, plante, chine, tige, base, tourte, cham...","Asperges vertes, eau, sel, acidifiant : acide ...","Aliments et boissons à base de végétaux, Alime...",vegetables,248.000000,4.600000,16.000000,22.000000
136633,189744,Mesclun (5/6 portions),"base, 5-6, san, portion, florette, frai, verte...",Mesclun (mélange de jeunes pousses comprenant ...,"Aliments et boissons à base de végétaux, Alime...",vegetables,18.000000,2.200000,0.500000,0.500000
225105,324656,Betteraves Rouges Entières,"origine, entiere, base, fruit, betterave, de, ...",,"Aliments et boissons à base de végétaux, Alime...",vegetables,52.000000,0.900000,0.400000,11.000000
39352,59364,Young sweet corn in brine,"coconut, corn, based, in, co, beverage, food, ...","Young sweet corn, water, salt, citric acid (as...","Plant-based foods and beverages, Plant-based f...",vegetables,25.000000,1.670000,0.000000,5.000000
155629,221229,salsifis,"vegetaux, origine, made-in-france, et, en, con...",,"Aliments et boissons à base de végétaux, Alime...",vegetables,28.000000,1.100000,0.200000,4.100000


In [21]:
import requests

In [46]:
r = search_image('Cake aux légumes')

In [47]:
r

['https://cac.img.pmdstatic.net/fit/http.3A.2F.2Fprd2-bone-image.2Es3-website-eu-west-1.2Eamazonaws.2Ecom.2Fcac.2F2018.2F09.2F25.2Fb48d4c3f-8d09-4cde-8761-b7f739a23300.2Ejpeg/750x562/quality/80/crop-from/center/cake-aux-legumes-d-ete-et-fromage-de-chevre.jpeg']

In [25]:
import json

{'_type': 'Images',
 'instrumentation': {'_type': 'ResponseInstrumentation'},
 'readLink': 'https://api.cognitive.microsoft.com/api/v7/images/search?q=test',
 'webSearchUrl': 'https://www.bing.com/images/search?q=test&FORM=OIIARP',
 'queryContext': {'originalQuery': 'test',
  'alterationDisplayQuery': 'test',
  'alterationOverrideQuery': '+test',
  'alterationMethod': 'AM_JustChangeIt',
  'alterationType': 'CombinedAlterationsChained'},
 'totalEstimatedMatches': 835,
 'nextOffset': 1,
 'currentOffset': 0,
 'value': [{'webSearchUrl': 'https://www.bing.com/images/search?view=detailv2&FORM=OIIRPO&q=test&id=9AE608EF8488887A0BE0157E4479858C84FDC669&simid=608018355506665184',
   'name': 'New antibody test can tell you if you have been infected ...',
   'thumbnailUrl': 'https://tse2.mm.bing.net/th?id=OIP.oZ05ftOllQ0vNjoyyaWKfwHaEJ&pid=Api',
   'datePublished': '2020-03-25T22:39:00.0000000Z',
   'isFamilyFriendly': True,
   'contentUrl': 'https://news.images.itv.com/image/file/2164058/img.jpg'

'{"_type": "Images", "instrumentation": {"_type": "ResponseInstrumentation"}, "readLink": "https:\\/\\/api.cognitive.microsoft.com\\/api\\/v7\\/images\\/search?q=test", "webSearchUrl": "https:\\/\\/www.bing.com\\/images\\/search?q=test&FORM=OIIARP", "queryContext": {"originalQuery": "test", "alterationDisplayQuery": "test", "alterationOverrideQuery": "+test", "alterationMethod": "AM_JustChangeIt", "alterationType": "CombinedAlterationsChained"}, "totalEstimatedMatches": 835, "nextOffset": 1, "currentOffset": 0, "value": [{"webSearchUrl": "https:\\/\\/www.bing.com\\/images\\/search?view=detailv2&FORM=OIIRPO&q=test&id=9AE608EF8488887A0BE0157E4479858C84FDC669&simid=608018355506665184", "name": "New antibody test can tell you if you have been infected ...", "thumbnailUrl": "https:\\/\\/tse2.mm.bing.net\\/th?id=OIP.oZ05ftOllQ0vNjoyyaWKfwHaEJ&pid=Api", "datePublished": "2020-03-25T22:39:00.0000000Z", "isFamilyFriendly": true, "contentUrl": "https:\\/\\/news.images.itv.com\\/image\\/file\\/21

In [8]:
df = df[df['product_name'].str.lower().str.contains('banan')]

In [9]:
df.to_sql('bananas',engine,if_exists='append',index=False)

inference

In [11]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from food.clipmodel import *
import os
os.environ["HF_DATASETS_OFFLINE"] = "2"
os.environ["TRANSFORMERS_OFFLINE"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from food.tools import *
from food.psql import *
from food.paths import *
from PIL import Image
from food.clipmodel import image2clip
import custom_pandas as cpd
from tqdm import tqdm


inference product name only

In [14]:
iteration = 1

In [15]:

query = f"""select id,product_name,keywords,ingredients_text,categories,food_groups
             from bananas
            where clip{iteration} is Null """

total = engine.execute(f'select count(*) from ({query}) a').one()

bs = 1
pd_iter = cpd.read_sql_query(query, engine, chunksize=bs, index_col='id')


In [ ]:

for inp in tqdm(pd_iter, desc="clip food inference", total=total[0] // bs):  
    try:
        text =inp.fillna("")
        for c in text.columns: text[c] = text[c].str.replace('NaN','')
        text = text['product_name']+ '. ' + text['food_groups']+ '. '+ text['categories']+ '. ' + text['ingredients_text']+'. ' + text['keywords'] 
        clip = text2clip(text.tolist()[0][:150]).numpy().tolist()
        inp['clip'] = [clip]
        insert_ignore(inp,'foods_big',update=True,update_cols=['clip'],unique_cols=['product_name'])
    except RuntimeError as e:
        print(e)

In [17]:
[df[c]+'. ' for c in df.columns]

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U2')) -> None

In [16]:
df

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g
38,664434,"Ethiquable Bananen Chips Scharf (2,34 Eur / 10...","34, bananen, scharf, 100, eur, chip, ethiquable",gswaarde per 100 g /Valen g/Durchschnittliche ...,NaN,NaN,529.0,2.2,31.0,60.0
74,665066,Smoothie Erdbeer-Banane-Traube,"no-added-sugar, away, erdbeer-banane-traube, t...","40% Erdbeermark , 26% Bananmark , 25% Traubens...","Getränke, Ungezuckerte Getränke",NaN,60.0,0.8,0.2,12.1
297,668761,3Bears Porridge Mohnige Banane,"und, porridge, in, vegan, getreide, lebensmitt...","61,5% _Vollkorn-Haferflocken_, 22% getrocknete...","Pflanzliche Lebensmittel und Getränke, Pflanzl...",breakfast-cereals,370.0,11.0,6.9,62.0
400,670314,LAC Fruchtjoghurt Banane,"milch, gluten-free, fruchtjoghurt, lac, joghur...","Lactosefreiero Joghurt, 18 % Bananen-Zubereitu...","Milchprodukte, Fermentierte Lebensmittel, Ferm...",milk-and-yogurt,104.0,4.3,2.9,14.5
430,670905,FruchtBar Granatapfel-Erdbeere-Pfirsich-Banane...,"smoothie, getränke, und, bio, ohne, granatapfe...","pürierter Pfirsich (60%), pürierte Erdbeere (1...","Pflanzliche Lebensmittel und Getränke, Getränk...",NaN,54.0,0.9,0.5,11.0
...,...,...,...,...,...,...,...,...,...,...
1110394,649319,Kirsche Banane,"zertifiziert, teebeutel, mit, heißgetränke, te...","Äpfel, Hibiskus, Hagebutten, Orangenschalen, A...","Pflanzliche Lebensmittel und Getränke, Getränk...",unknown,2.0,0.0,0.0,0.6
1110670,654088,Banane - Apfel,"riegel, bio, süßer, alnatura, banane, imbis, a...","Bananenflocken* 31%, Apfelsaftkonzentrat* 30%,...","Imbiss, Süßer Snack, Riegel",biscuits-and-cakes,368.0,5.0,8.6,63.9
1110785,656040,Nektar bananowy,"nektar, soki, auchan, napoje, roślin, owocowe,...","Woda, przecier bananowy, cukier, regulator kwa...","Żywność i napoje na bazie roślin, Napoje, Napo...",sweetened-beverages,45.0,0.5,0.5,10.0
1111184,662304,Lycka Banane Avocado Haselnuss,"gmbh, banane, süßstoff, kalorienarme, eistee, ...","Wasser, Avocadosoße* (15%) (Agavendicksaft*, A...","Getränke, Getränke mit Süßstoff, Kalorienarme ...",teas-and-herbal-teas-and-coffees,126.0,1.3,6.2,14.0
